In [2]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import numpy as np
import pandas as pd
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer

app = Flask(__name__)
CORS(app)

# Load the hotel data and perform necessary preprocessing
clear_trip = pd.read_csv("C:/Users/bhopp/OneDrive/Pictures/Saved Pictures/cleartrip_com-travel_sample_final_working.csv")
clear_trip['Price_per_night(Rs)'] = np.random.randint(500, 20000, size=len(clear_trip))
del clear_trip['cleartrip_seller_rating']
del clear_trip['country']
del clear_trip['crawl_date']
del clear_trip['image_count']
del clear_trip['image_urls']
del clear_trip['latitude']
del clear_trip['longitude']
del clear_trip['qts']
del clear_trip['room_area']
del clear_trip['sitename']
del clear_trip['uniq_id']

# Function to calculate the number of guests based on room type
room_no = [
    ('Standard', 2),
    ('Premium', 2),
    ('Business', 3),
    ('Deluxe', 3),
    ('Super Deluxe', 4),
    ('Royal Suite', 2),
    ('Deluxe', 4),
    ('Economy', 4),
    ('Executive', 2),
    ('Superior', 1),
    ('Tent', 2),
    ('AC Cottage', 4),
    ('Double Bedroom', 2),
    ('standard non a/c room', 2),
    ('single', 4),
    ('village stay', 1),
    ('Jodhpur Deluxe', 1),
    ('Chakra Cottages', 1),
    ('Village Stay', 2),
    ('Garden Cottage', 2)
]

def calc():
    guests_no = []
    for i in range(clear_trip.shape[0]):
        temp = str(clear_trip['room_type'][i]).lower().split()
        flag = 0
        for j in range(len(temp)):
            for k in range(len(room_no)):
                if temp[j] == room_no[k][0]:
                    guests_no.append(room_no[k][1])
                    flag = 1
                    break
            if flag == 1:
                break
        if flag == 0:
            guests_no.append(2)
    clear_trip['guests_no'] = guests_no

calc()

# Add a route to serve the index.html file at the root URL
@app.route('/')
def serve_index():
    return send_file('websitedraft.html')

# Endpoint for city-based hotel recommendations
@app.route('/recommendations/city', methods=['POST'])
def city_based_recommendations():
    city = request.json.get('city')

    clear_trip['city'] = clear_trip['city'].str.lower()
    city_base = clear_trip[clear_trip['city'] == city.lower()]
    city_base = city_base.sort_values(by='hotel_star_rating', ascending=False)
    city_base.drop_duplicates(subset='property_id', keep='first', inplace=True)

    if city_base.empty:
        return jsonify(message='No Hotels Available')
    else:
        ctrip = city_base[['property_name', 'hotel_star_rating', 'address', 'Price_per_night(Rs)', 'room_count',
                           'room_facilities', 'room_type', 'Room_availability', 'hotel_description']]
        return jsonify(ctrip.head().to_dict(orient='records'))

# Endpoint for room-based hotel recommendations
@app.route('/recommendations/room', methods=['POST'])
def room_based_recommendations():
    city = request.json.get('city')
    room_type = request.json.get('roomtype')

    clear_trip['room_type'] = clear_trip['room_type'].str.lower()
    clear_trip['city'] = clear_trip['city'].str.lower()
    room_base = clear_trip[(clear_trip['city'] == city.lower()) & (clear_trip['room_type'] == room_type.lower())]
    room_base = room_base.sort_values(by='hotel_star_rating', ascending=False)
    room_base.drop_duplicates(subset='property_id', keep='first', inplace=True)

    if room_base.empty:
        return jsonify(message='No Hotels Available')
    else:
        ctrip = room_base[['property_name', 'city', 'hotel_star_rating', 'address', 'Price_per_night(Rs)', 'room_count',
                           'room_facilities', 'room_type', 'Room_availability', 'hotel_description']]
        return jsonify(ctrip.head().to_dict(orient='records'))

# Endpoint for price-based hotel recommendations
@app.route('/recommendations/price', methods=['POST'])
def price_based_recommendations():
    city = request.json.get('city')
    price = request.json.get('price')

    clear_trip['city'] = clear_trip['city'].str.lower()
    price_base = clear_trip[(clear_trip['city'] == city.lower()) & (clear_trip['Price_per_night(Rs)'] <= price)]
    price_base = price_base.sort_values(by='hotel_star_rating', ascending=False)
    price_base.drop_duplicates(subset='property_id', keep='first', inplace=True)

    if price_base.empty:
        return jsonify(message='No Hotels Available')
    else:
        ctrip = price_base[['property_name', 'city', 'hotel_star_rating', 'address', 'Price_per_night(Rs)', 'room_count',
                            'room_facilities', 'room_type', 'Room_availability', 'hotel_description']]
        return jsonify(ctrip.head().to_dict(orient='records'))


# Endpoint for popular city-based hotel recommendations
@app.route('/recommendations/popular/city', methods=['POST'])
def popular_city_based_recommendations():
    city = request.json.get('city')
    room_type = request.json.get('roomtype')
    price = request.json.get('price')
    number = request.json.get('number')
    availability = request.json.get('availability')

    clear_trip['city'] = clear_trip['city'].str.lower()
    clear_trip['room_type'] = clear_trip['room_type'].str.lower()
    pop_based = clear_trip[(clear_trip['city'] == city.lower()) & (clear_trip['room_type'] == room_type.lower()) & 
                           (clear_trip['Price_per_night(Rs)'] <= price) & 
                           (clear_trip['guests_no'] == number) & 
                           (clear_trip['Room_availability'] >= availability)]
    pop_based = pop_based.sort_values(by='hotel_star_rating', ascending=False)
    pop_based.drop_duplicates(subset='property_id', keep='first', inplace=True)

    if pop_based.empty:
        return jsonify(message='Sorry No Hotels Available\n Tune your constraints')
    else:
        ctrip = pop_based[['property_name', 'room_type', 'guests_no', 'hotel_star_rating', 'address', 'Price_per_night(Rs)',
                           'Room_availability', 'room_facilities']]
        return jsonify(ctrip.head(10).to_dict(orient='records'))

# Endpoint for requirement-based hotel recommendations
@app.route('/recommendations/requirement', methods=['POST'])
def requirement_based_recommendations():
    city = request.json.get('city')
    number = request.json.get('number')
    room_type = request.json.get('roomtype')
    availability = request.json.get('availability')
    price = request.json.get('price')
    features = request.json.get('features')

    clear_trip['city'] = clear_trip['city'].str.lower()
    clear_trip['room_type'] = clear_trip['room_type'].str.lower()
    clear_trip['room_facilities'] = clear_trip['room_facilities'].str.lower()
    features = features.lower()
    features_tokens = word_tokenize(features)  
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    f1_set = {w for w in features_tokens if not w in sw}
    f_set = set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    req_based = clear_trip[(clear_trip['city'] == city.lower()) & (clear_trip['guests_no'] == number) & 
                           (clear_trip['Price_per_night(Rs)'] <= price) & 
                           (clear_trip['Room_availability'] >= availability) & 
                           (clear_trip['room_type'] == room_type.lower())]
    req_based = req_based.set_index(np.arange(req_based.shape[0]))
    cos = []
    for i in range(req_based.shape[0]):
        temp_tokens = word_tokenize(req_based['room_facilities'][i])
        temp1_set = {w for w in temp_tokens if not w in sw}
        temp_set = set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        cos.append(len(rvector))
    req_based['similarity'] = cos
    req_based = req_based.sort_values(by='similarity', ascending=False)
    req_based.drop_duplicates(subset='property_id', keep='first', inplace=True)

    return jsonify(req_based[['property_name', 'room_type', 'guests_no', 'hotel_star_rating', 'address', 
                              'room_facilities', 'hotel_facilities', 'Price_per_night(Rs)', 
                              'Room_availability', 'similarity']].head(10).to_dict(orient='records'))

# Endpoint for collaborative filtering-based hotel recommendations
@app.route('/recommendations/collaborative', methods=['POST'])
def collaborative_filtering_recommendations():
    user_id = request.json.get('user_id')
    property_id = request.json.get('property_id')
    ratingscollection = clear_trip.pivot_table(index='user_id', columns='property_id', values='user_rating')
    ratingscollection.head(5)
    ratingscollection = ratingscollection.apply(lambda row: row.fillna(row.mean()), axis=1)
    ratingscollection.head(5)
    usercorrelation = ratingscollection.T.corr()
    usercorrelation.head(5)
    
    def cf(user_id, property_id):
        user1corr = usercorrelation.iloc[user_id - 1]
        user1corr.sort_values(ascending=False, inplace=True)
        user1corr.dropna(inplace=True)
        hundreduserslikeuser1 = user1corr[1:101].keys()
        count = 0
        users = list()
        for user in hundreduserslikeuser1:
            if clear_trip[(clear_trip['user_id'] == user) & (clear_trip['property_id'] == property_id)]['user_rating'].sum():
                count += 1
                users.append(user)
        sum_similarity = 0
        weighted_ratings = 0
        for user in users:
            weighted_ratings += user1corr[1:101].loc[user] * clear_trip[(clear_trip['user_id'] == user) & 
                                                                        (clear_trip['property_id'] == property_id)]['user_rating'].sum()
            sum_similarity += user1corr[1:101].loc[user]
        estimated_rating = (weighted_ratings / sum_similarity)
        
        if estimated_rating >= 2:
            property_name = clear_trip.loc[clear_trip['property_id'] == property_id, 'property_name'].iloc[0]
            return jsonify(property_name=property_name, estimated_rating=estimated_rating)
        else:
            return jsonify(estimated_rating=estimated_rating)
    
    return cf(user_id, property_id)


if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Aug/2023 16:28:26] "OPTIONS /recommendations/requirement HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2023 16:28:26] "OPTIONS /recommendations/city HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2023 16:28:26] "OPTIONS /recommendations/room HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2023 16:28:27] "POST /recommendations/requirement HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2023 16:28:27] "POST /recommendations/city HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2023 16:28:27] "POST /recommendations/room HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2023 16:29:08] "OPTIONS /recommendations/requirement HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2023 16:29:08] "OPTIONS /recommendations/city HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2023 16:29:08] "OPTIONS /recommendations/room HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2023 16:29:08] "POST /recommendations/requirement HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2023 16:29:08] "POST /recommendations/city HTTP/1.1" 200 -
127.0.